# Time-lapse animation (GIF) of NDVI change over Singapore from 2016 to 2023
# using Sentinel-2 imagery in Earth Engine + Google Colab.

In [2]:
# ===============================================
# NDVI Time-lapse for Singapore using Sentinel-2
# ===============================================

# --- Install required libraries ---
!pip -q install --upgrade earthengine-api geemap folium pandas matplotlib

# --- Import Python libraries ---
import ee, pandas as pd, matplotlib.pyplot as plt, geemap
import os
from google.colab import userdata

# --- Authenticate with Google Earth Engine ---
PROJECT_ID = userdata.get('GCP_ID')  # Fetch GCP project ID stored securely in Colab
ee.Authenticate()
ee.Initialize(project=PROJECT_ID)

# ===============================================
# SECTION 1: Setup study region — Singapore
# ===============================================

# Load global administrative boundaries (level0 = countries)
admin_boundaries = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level0")

# Filter to only Singapore
singapore = admin_boundaries.filter(
    ee.Filter.eq('ADM0_NAME', 'Singapore')
).geometry()

# Print area (for curiosity / debugging)
print("Singapore area (sq km):", singapore.area().divide(1e6).getInfo())
# --- TRY changing the country name to 'India' or 'Nepal' or any ADM0_NAME ---


# ===============================================
# SECTION 2: Define helper functions
# ===============================================

# --- Cloud masking function for Sentinel-2 ---
# Removes pixels that have clouds or cirrus based on QA60 band bits.
def s2_cloud_mask(img):
    qa = img.select('QA60')  # QA60 band contains cloud mask info
    cloud_bit  = 1 << 10     # bit 10 = clouds
    cirrus_bit = 1 << 11     # bit 11 = cirrus
    mask = qa.bitwiseAnd(cloud_bit).eq(0).And(qa.bitwiseAnd(cirrus_bit).eq(0))
    return img.updateMask(mask)  # mask out cloudy pixels


# --- Add NDVI band ---
def add_ndvi(img):
    # NDVI = (NIR - Red) / (NIR + Red)
    ndvi = img.normalizedDifference(['B8', 'B4']).rename('NDVI')  # B8=NIR, B4=Red
    return img.addBands(ndvi)

# ===============================================
# SECTION 3: Define dataset and date range
# ===============================================

start_year, end_year = 2016, 2023   # --- change these years to expand or shrink the timeline ---

# Load Sentinel-2 surface reflectance collection
s2 = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
      .filterBounds(singapore)  # keep only images over Singapore
      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 60))  # only <60% cloudy scenes
      .map(s2_cloud_mask)  # apply custom cloud mask
      .map(add_ndvi))      # compute NDVI

# ===============================================
# SECTION 4: Annual NDVI → RGB images for video
# ===============================================

# Build a list of years
years = ee.List.sequence(start_year, end_year)

# Function to return an RGB image for a given year using NDVI palette
def yearly_ndvi_rgb(y):
    y = ee.Number(y)
    start = ee.Date.fromYMD(y, 1, 1)
    end   = start.advance(1, 'year')

    # Filter to current year, compute median NDVI
    ndvi_img = s2.filterDate(start, end).select('NDVI').median()

    # Visualize NDVI as RGB image using color palette
    vis_img = ndvi_img.visualize(
        min=0, max=1,
        palette=['#d73027','#fee08b','#1a9850']  # red-yellow-green NDVI scale
    )

    # Make sure output has 3 RGB bands. If not (e.g., no images), return a blank image
    band_count = vis_img.bandNames().size()
    blank_rgb = ee.Image.constant([0,0,0]).rename(['R','G','B']).uint8()

    # If band count ≠ 3 (e.g., empty image), use blank. Otherwise use vis_img.
    vis_rgb = ee.Image(ee.Algorithms.If(
        band_count.eq(3),
        vis_img.rename(['R','G','B']).unmask(0),
        blank_rgb
    ))

    # Tag image with timestamp so Earth Engine knows its year
    return vis_rgb.set('system:time_start', start.millis())

# Apply this function to all years — creates one RGB image per year
annual_ndvi_ic = ee.ImageCollection(years.map(yearly_ndvi_rgb))

# ===============================================
# SECTION 5: Export as a time-lapse video
# ===============================================

# Region to export: bounding box of Singapore
region_bounds = singapore.bounds().getInfo()['coordinates']

# Define video parameters
video_params = {
    'dimensions': 1080,               # output resolution
    'region': region_bounds,          # spatial bounds
    'framesPerSecond': 1              # 1 frame per year → one frame per second
}

# Get Earth Engine video URL (GIF-compatible .mp4)
import requests
video_url = annual_ndvi_ic.getVideoThumbURL(video_params)
print("Video URL:", video_url)

# Download MP4 using requests
mp4_path = "singapore_ndvi.mp4"
r = requests.get(video_url)
with open(mp4_path, "wb") as f:
    f.write(r.content)
print("MP4 saved:", mp4_path)


# ===============================================
# SECTION 6: Convert MP4 to GIF with year labels
# ===============================================

# Install video + image processing libraries
!pip -q install imageio imageio-ffmpeg pillow

# Import tools to read video, draw text, save GIF
import imageio.v3 as iio
from PIL import Image as PILImage, ImageDraw, ImageFont
from IPython.display import Image as IPyImage

# Read MP4 as list of frames
frames = iio.imread(mp4_path)
labeled_frames = []

# Add year labels to each frame (bottom left corner)
for i, frame in enumerate(frames):
    img = PILImage.fromarray(frame)
    draw = ImageDraw.Draw(img)
    try:
        font = ImageFont.truetype("DejaVuSans-Bold.ttf", 48)
    except:
        font = ImageFont.load_default()
    year_text = str(start_year + i)
    draw.text((20, img.height - 70), year_text, font=font,
              fill="white", stroke_width=3, stroke_fill="black")
    labeled_frames.append(img)

# Save as animated GIF
gif_path = "singapore_ndvi.gif"
labeled_frames[0].save(gif_path, save_all=True, append_images=labeled_frames[1:],
                       duration=1000, loop=0)  # 1s per frame, loop forever

# Display in notebook
IPyImage(filename=gif_path)

Output hidden; open in https://colab.research.google.com to view.